<a href="https://colab.research.google.com/github/hyuna0926/cp2_phase2/blob/main/recommend/tunning_implicit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 필요 라이브러리 불러오기

In [1]:
! pip install implicit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.6/18.6 MB 44.0 MB/s eta 0:00:00


In [2]:
import pandas as pd
from google.colab import drive
from sklearn.model_selection import train_test_split
from scipy import sparse
from scipy.sparse import csr_matrix
from tqdm.notebook import tqdm
import numpy as np

from implicit.als import AlternatingLeastSquares as ALS
import implicit
import random
import os

/usr/local/lib/python3.8/dist-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: no CUDA-capable device is detected (/project/./implicit/gpu/utils.h:71)'
  warnings.warn(


In [5]:
# implicit 라이브러리에서 권장하고 있는 부분입니다. 
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

# 2. 데이터 불러오기 및 분리하기


In [3]:
# add_to_cart까지 포함한것
df = pd.read_parquet('/content/drive/MyDrive/CP2_Phase2/cart_purchase.parquet')

In [ ]:
df.head()

,created_at,customer_id,product_id,p_idx,event,c_idx
0,2018-07-29 15:22:01,5868,54728,29317,purchase,3039
1,2021-01-15 19:52:40,82831,54728,29317,purchase,41968
2,2016-10-20 10:16:35,82831,34982,8394,purchase,41968
3,2018-08-26 14:09:52,82831,8627,9052,purchase,41968
4,2018-03-17 15:08:08,82831,46703,23994,purchase,41968


In [ ]:
df.shape

(1876781, 6)

In [ ]:
df['customer_id'].nunique()

50698

In [ ]:
df['p_idx'].nunique()

31121

- 구매했으니 1로 둠

In [4]:
df['values']=1

In [ ]:
df.head()

,created_at,customer_id,product_id,p_idx,event,c_idx,values
0,2018-07-29 15:22:01,5868,54728,29317,purchase,3039,1
1,2021-01-15 19:52:40,82831,54728,29317,purchase,41968,1
2,2016-10-20 10:16:35,82831,34982,8394,purchase,41968,1
3,2018-08-26 14:09:52,82831,8627,9052,purchase,41968,1
4,2018-03-17 15:08:08,82831,46703,23994,purchase,41968,1


- groupby로 만들어서 train, test customer_id 비율을 맞춰줬음
  - 비율이 안맞아서 검증셋에 없는 customer_id가 있지만 그래도 균등하게 나눠진듯

In [6]:
df_group=df.groupby(['c_idx'], as_index = False).count()
upper_20 = list(df_group.query("values >= 20").c_idx) #22403
lower_20 = list(df_group.query("values < 20").c_idx) #28295

In [7]:
df_group=df.groupby(['c_idx'], as_index = False).count()

upper = df[df['c_idx'].isin(df_group.query("values >= 20").c_idx)]


test = upper.groupby('c_idx').sample(frac=0.2,random_state=42)
train = upper.drop(test.index)

user_20 = sparse.csr_matrix((train['values'],(train['c_idx'], train['p_idx'])))
test_user_20 = sparse.csr_matrix((test['values'],(test['c_idx'], test['p_idx'])))

In [8]:
# 검증셋 만들기
test_df = test.groupby('c_idx')['p_idx'].unique().to_frame().reset_index()
test_df

,c_idx,p_idx
0,0,"[3393, 14555, 9702, 25009, 24439, 19617, 8865,..."
1,4,"[13807, 1213, 3051, 23329]"
2,6,"[20858, 27397, 18336, 5254, 30369, 3425, 889, ..."
3,7,"[28436, 438, 8041, 16482, 17893, 27284, 10339,..."
4,9,"[29679, 998, 7848, 26990, 25655, 22110, 11228]"
...,...,...
22398,50689,"[7186, 12438, 23458, 19032, 25009, 11686, 5743..."
22399,50690,"[3853, 14872, 29576, 14140, 243, 2125, 21300, ..."
22400,50693,"[22738, 886, 22855, 23942, 2454, 25744]"
22401,50694,"[8074, 18765, 571, 21249, 784, 1870, 16300, 83..."


# 3. implicit 라이브러리 튜닝함수

In [ ]:
def tunning_pre_hit(user_items,params,n_iters=10):
  results =[]
  for _ in tqdm(range(n_iters)):
    factor = np.random.choice(params['factor'])
    alpha = np.random.choice(params['alpha'])
    regularization = np.random.choice(params['regularization'])
    iteration = np.random.choice(params['iteration'])

    ALS = implicit.als.AlternatingLeastSquares(factors=factor, alpha=alpha, regularization=regularization, iterations=iteration,
              random_state=42, calculate_training_loss=True)
    ALS.fit(user_items)

    #hit_rate, precision
    total= 0 # hit
    user = 0
    precision=0
    for c_idx in range(500):
      count=0 # for문 돌 때마다 리셋
      if c_idx in upper_20:
        recommend = ALS.recommend(c_idx, user_20[c_idx],N=25)[0]
        buy_test = test_df[test_df['c_idx']==c_idx].p_idx.values[0]
        user+=1
        for i in buy_test:
          for j in recommend:
            if i==j:
              count+=1
              precision+=1 #precision
      if count >=1: #count가 1 이상이면 hit한 것이니까 전체에 1 추가
        total+=1
      
    hit_rate = total/user
    precision_mean = (precision/25)/user

    
    result = [factor, alpha, regularization, iteration, hit_rate, precision_mean]
    results.append(result)
  frame = pd.DataFrame(results, columns=['factor','alpha','regularization','iteration','hit_rate','precision'])
  frame = frame.sort_values('precision', ascending=False)[:5]

  return frame

In [ ]:
params = {
    'factor': np.arange(90,210,10),
    'alpha' : np.arange(1,21,5),
    'regularization' : np.arange(1,41,5),
    'iteration' : np.arange(10,31,5)
}

In [ ]:
tunning_pre_hit(user_20,params,n_iters=5)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

,factor,alpha,regularization,iteration,hit_rate,precision
0,110,16,26,20,0.182203,0.008983
1,110,11,6,20,0.177966,0.008475
4,160,16,1,20,0.169492,0.008305
2,90,6,6,30,0.165254,0.008136
3,160,11,26,15,0.139831,0.007966


In [ ]:
params1 = {
    'factor': np.arange(90,210,10),
    'alpha' : np.arange(1,21,4),
    'regularization' : np.arange(1,30,5),
    'iteration' : np.arange(15,31,5)
}
tunning_pre_hit(user_20,params1,n_iters=5)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

,factor,alpha,regularization,iteration,hit_rate,precision
1,170,1,26,30,0.220339,0.013390
3,90,9,16,30,0.190678,0.009661
0,160,5,1,25,0.161017,0.009153
2,160,13,11,20,0.169492,0.008475
4,170,13,21,25,0.173729,0.007797


## grid

In [16]:
def tunning_grid(user_items,params,n_iters=10):
  results =[]
  for factor in tqdm(params['factor']):
    for alpha in params['alpha']:
      for regularization in params['regularization']:
        ALS = implicit.als.AlternatingLeastSquares(factors=factor, alpha=alpha, regularization=regularization, iterations=20,
              random_state=42, calculate_training_loss=True)
        ALS.fit(user_items)

        #hit_rate, precision
        total= 0 # hit
        user = 0
        precision=0
        for c_idx in range(5000):
          count=0 # for문 돌 때마다 리셋
          if c_idx in upper_20:
            recommend = ALS.recommend(c_idx, user_20[c_idx],N=25)[0]
            buy_test = test_df[test_df['c_idx']==c_idx].p_idx.values[0]
            user+=1
            for i in buy_test:
              for j in recommend:
                if i==j:
                  count+=1
                  precision+=1 #precision
          if count >=1: #count가 1 이상이면 hit한 것이니까 전체에 1 추가
            total+=1
      
        hit_rate = total/user
        precision_mean = (precision/25)/user

        
        result = [factor, alpha, regularization,  hit_rate, precision_mean]
        results.append(result)
  frame = pd.DataFrame(results, columns=['factor','alpha','regularization','hit_rate','precision'])
  frame = frame.sort_values('precision', ascending=False)[:5]

  return frame

In [17]:
params_grid = {
    'factor': [160,180,200],
    'alpha' : [1,3,5],
    'regularization' : [28,30,32],
    # 'iteration' : np.arange(15,31,5)
}
tunning_grid(user_20,params_grid)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

,factor,alpha,regularization,hit_rate,precision
11,180,1,32,0.282189,0.016061
1,160,1,30,0.280843,0.016043
2,160,1,32,0.280395,0.016025
20,200,1,32,0.279498,0.015953
10,180,1,30,0.280395,0.015917


# 4. 최적의 하이퍼파리미터

In [ ]:
#최적
ALS = implicit.als.AlternatingLeastSquares(factors=200, alpha=1, regularization=31, iterations=30,
              random_state=42, calculate_training_loss=True)

ALS.fit(user_20)

  0%|          | 0/30 [00:00<?, ?it/s]

In [ ]:
# als추천과 data셋에 있는 p_idx 비교
def precision_at_k(c_idx,k=25):
  
  count=0
  buy_test = test_df[test_df['c_idx']==c_idx].p_idx.values[0]
  recommendations_t = ALS.recommend(c_idx, user_20[c_idx],N=k ,filter_already_liked_items=False)
  for i in buy_test:
    for j in recommendations_t[0]:
      if i==j:
        count+=1
  
  precision = count/len(recommendations_t[0])
  return precision

In [ ]:
def mean_precision_k(k):
  precision=0
  total =0
  for i in range(1,k+1):
    #train,test셋으로 분리했기 때문에 없는 user_id는 넘김
    if i in upper_20: #20개 이상 산 사람
      total+=1
      precision+=precision_at_k(i,25)
    else:
      pass
  mean_precision = precision/total
  return mean_precision

In [ ]:
mean_precision_k(500)

0.011744680851063834

In [ ]:
mean_precision_k(49000)

0.01389063365970847

In [ ]:
def hit_rate(k):
  total= 0
  user = 0
  for c_idx in range(k):
    count=0 # for문 돌 때마다 리셋
    if c_idx in upper_20:
      recommend = ALS.recommend(c_idx, user_20[c_idx],N=25)[0]
      buy_test = test_df[test_df['c_idx']==c_idx].p_idx.values[0]
      user+=1
      for i in buy_test:
        for j in recommend:
          if i==j:
            count+=1
    if count >=1: #count가 1 이상이면 hit한 것이니까 전체에 1 추가
      total+=1
    
  hit_rate = total/user

  return hit_rate

In [ ]:
hit_rate(5000)

0.27635711081202335

- 엇 hit_rate 더 높게 나옴

In [ ]:
# 한번 도전
ALS1 = implicit.als.AlternatingLeastSquares(factors=170, alpha=1, regularization=30, iterations=20,
              random_state=42, calculate_training_loss=True)

ALS1.fit(user_20)

  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
# als추천과 data셋에 있는 p_idx 비교
def precision_at_k(c_idx,k=25):
  
  count=0
  buy_test = test_df[test_df['c_idx']==c_idx].p_idx.values[0]
  recommendations_t = ALS1.recommend(c_idx, user_20[c_idx],N=k ,filter_already_liked_items=False)
  for i in buy_test:
    for j in recommendations_t[0]:
      if i==j:
        count+=1
  
  precision = count/len(recommendations_t[0])
  return precision

In [ ]:
def mean_precision_k(k):
  precision=0
  total =0
  for i in range(1,k+1):
    #train,test셋으로 분리했기 때문에 없는 user_id는 넘김
    if i in upper_20: #20개 이상 산 사람
      total+=1
      precision+=precision_at_k(i,25)
    else:
      pass
  mean_precision = precision/total
  return mean_precision

In [ ]:
def hit_rate(k):
  total= 0
  user = 0
  for c_idx in range(k):
    count=0 # for문 돌 때마다 리셋
    if c_idx in upper_20:
      recommend = ALS1.recommend(c_idx, user_20[c_idx],N=25)[0]
      buy_test = test_df[test_df['c_idx']==c_idx].p_idx.values[0]
      user+=1
      for i in buy_test:
        for j in recommend:
          if i==j:
            count+=1
    if count >=1: #count가 1 이상이면 hit한 것이니까 전체에 1 추가
      total+=1
    
  hit_rate = total/user

  return hit_rate

In [ ]:
mean_precision_k(49000)

0.013539626824312121

In [ ]:
hit_rate(49000)

0.27969702567892113

- 3번째 도전

In [9]:
ALS2 = implicit.als.AlternatingLeastSquares(factors=180, alpha=1, regularization=29, iterations=20,
              random_state=42, calculate_training_loss=True)

ALS2.fit(user_20)

  0%|          | 0/20 [00:00<?, ?it/s]

In [10]:
# als추천과 data셋에 있는 p_idx 비교
def precision_at_k(c_idx,k=25):
  
  count=0
  buy_test = test_df[test_df['c_idx']==c_idx].p_idx.values[0]
  recommendations_t = ALS2.recommend(c_idx, user_20[c_idx],N=k ,filter_already_liked_items=False)
  for i in buy_test:
    for j in recommendations_t[0]:
      if i==j:
        count+=1
  
  precision = count/len(recommendations_t[0])
  return precision

In [11]:
def mean_precision_k(k):
  precision=0
  total =0
  for i in range(1,k+1):
    #train,test셋으로 분리했기 때문에 없는 user_id는 넘김
    if i in upper_20: #20개 이상 산 사람
      total+=1
      precision+=precision_at_k(i,25)
    else:
      pass
  mean_precision = precision/total
  return mean_precision

In [12]:
def hit_rate(k):
  total= 0
  user = 0
  for c_idx in range(k):
    count=0 # for문 돌 때마다 리셋
    if c_idx in upper_20:
      recommend = ALS2.recommend(c_idx, user_20[c_idx],N=25)[0]
      buy_test = test_df[test_df['c_idx']==c_idx].p_idx.values[0]
      user+=1
      for i in buy_test:
        for j in recommend:
          if i==j:
            count+=1
    if count >=1: #count가 1 이상이면 hit한 것이니까 전체에 1 추가
      total+=1
    
  hit_rate = total/user

  return hit_rate

In [13]:
mean_precision_k(49000)

0.013024200997598477

In [14]:
hit_rate(49000)

0.2758636615555145